[Wide-ResNet MixMatch](#scrollTo=NMzmcxDzLgdv&line=3&uniqifier=1)

[U_net MixMatch](#scrollTo=zPibROeofH1D&line=10&uniqifier=1)

[ResNet18 MixMatch](#scrollTo=JYqDLKwkwbdS&line=9&uniqifier=1)

[Wide-ResNet MixMatch + increasing epoch + increasing early stopping count + decreasing learning rate](#scrollTo=9gDzmZ03yrDs&line=7&uniqifier=1)


[Wide-ResNet MixMatch + increasing epoch + increasing early stopping count + decreasing learning rate + Weight Decay ](#scrollTo=EtxH5QRz4M4X&line=4&uniqifier=1)

In [1]:
from __future__ import print_function

!pip install progress

import easydict
import os
import shutil
import time
import random

import numpy as np

import torch
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.optim as optim
import torch.utils.data as data
import torchvision.transforms as transforms
import torch.nn.functional as F


import os
import sys
# 다른 py 파일을 import하기 위한 code 입니다. Google Drive 상에서 작동하게 하기 위한 코드로, /Colab Notebooks/20180180_task_2'가 제 환경에서의 개인적인 경로입니다.
from google.colab import drive
drive.mount('/content/gdrive')
sys.path.insert(0, '/content/gdrive/My Drive/Colab Notebooks/20180180_task_2')


# 아래 import 파일들은 모두 제공된 skeleton 입니다.
import models.wideresnet as models
import models.loss as loss
import dataset.cifar10 as dataset
from utils import Bar, Logger, AverageMeter, accuracy, mkdir_p, savefig



  Created wheel for progress: filename=progress-1.5-cp36-none-any.whl size=8074 sha256=794784d3fff60fa1f9d76c7489bc6c718536655a7109e13876fded49f16b5146
  Stored in directory: /root/.cache/pip/wheels/6c/c8/80/32a294e3041f006c661838c05a411c7b7ffc60ff939d14e116
Successfully built progress
Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [2]:
# 주어진 for_student.ipynb 의 기본 skeleton입니다.

## This is argument.
args = easydict.EasyDict({
    "epochs": 16,   # number of total epochs to run
    "batch_size": 64, # train batchsize
    "lr": 0.002,       # initial learning rate
    "resume": '',     # path to latest checkpoint (default: none)
    "gpu": 0,
    "T" : 0.5,
    "alpha" : 0.75,
    "n_labeled": 250 , # Number of labeled data # please use 250 , 1000, 4000, 10000, 40000
    "val_iteration": 1024, # period of computing validation error
    "out": '', # Directory to output the result
    # You can add any configuration parameters for your own design!!
})

print(args)

{'epochs': 16, 'batch_size': 64, 'lr': 0.002, 'resume': '', 'gpu': 0, 'T': 0.5, 'alpha': 0.75, 'n_labeled': 250, 'val_iteration': 1024, 'out': ''}


In [0]:
# 주어진 for_student.ipynb의 기본 skeleton 에서 일부를 수정하였습니다.
# interleave 함수를 제거하고,  mixmatch를 수행하기 위한 함수를 정의하였습니다.
# mixmatch 함수는 https://github.com/gan3sh500/mixmatch-pytorch/blob/master/notebook.ipynb 의 코드를 바탕으로
# 일부 수정하여 완성하였습니다.

# mixmatch 함수를 기존 skeleton에 맞게 사용하기 위해 parameter로 unlabeled data를 합치치 않은 채로
# 2 set을 전달받고, 이를 torch.cat으로 합친 후 qb를 계산하였습니다.

def sharpen(x, T = 0.5):   # Sharpening을 수행하는 함수
  pt = x**(1/T)
  target_u = pt/pt.sum(dim=1,keepdim=True)
  target_u = target_u.detach()
  return target_u

def mixup(x1, p1, x2, p2, a = 0.75):  # mixup을 수행하는 함수
  Lambda = np.random.beta(a, a)
  Lambda = max(Lambda, 1 - Lambda)  

  x = Lambda * x1 + (1 - Lambda) * x2
  y = Lambda * p1 + (1 - Lambda) * p2
  return x, y

def mixmatch(xb, y, inputs_u, inputs_u2, model, K=2):    # K의 default값을 2로 지정하였음
    outputs_u = model(inputs_u)
    outputs_u2 = model(inputs_u2)
    ub = torch.cat([inputs_u, inputs_u2], dim = 0)
    p = (outputs_u + outputs_u2) / 2
    qb = sharpen(p)
    Ux = ub
    Uy = torch.cat([qb for _ in range(K)], dim=0)

    indices = np.random.shuffle(np.arange(len(xb) + len(Ux)))
    Wx = torch.cat([Ux, xb], axis=0)[indices]
    Wy = torch.cat([Uy, y], axis=0)[indices]

    X, p = mixup(xb, y, Wx[0][:len(xb)], Wy[0][:len(xb)])

    U, q = mixup(ub, Uy, Wx[0][len(xb):], Wy[0][len(xb):])

    return X, p, U, q


def train(labeled_trainloader, unlabeled_trainloader, model, optimizer, criterion, epoch, use_cuda=True):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    losses_x = AverageMeter()
    losses_u = AverageMeter()
    ws = AverageMeter()
    end = time.time()

    bar = Bar('Training', max=args.val_iteration)
    labeled_train_iter = iter(labeled_trainloader)
    unlabeled_train_iter = iter(unlabeled_trainloader)

    model.train()
    for batch_idx in range(args.val_iteration):
        try:
            inputs_x, targets_x = labeled_train_iter.next()
        except:
            labeled_train_iter = iter(labeled_trainloader)
            inputs_x, targets_x = labeled_train_iter.next()

        try:
            (inputs_u, inputs_u2), _ = unlabeled_train_iter.next()
        except:
            unlabeled_train_iter = iter(unlabeled_trainloader)
            (inputs_u, inputs_u2), _ = unlabeled_train_iter.next()

        # measure data loading time
        data_time.update(time.time() - end)

        batch_size = inputs_x.size(0)

        # Transform label to one-hot
        targets_x = torch.zeros(batch_size, 10).scatter_(1, targets_x.view(-1,1), 1)

        if use_cuda:
            inputs_x, targets_x = inputs_x.cuda(), targets_x.cuda(non_blocking=True)
            inputs_u = inputs_u.cuda()
            inputs_u2 = inputs_u2.cuda()
       
        # 아래 과정은 MixMatch가 일어나는 과정으로, 위에서 정의한 mixmatch, mixup, sharpen 함수를 사용합니다.
        with torch.no_grad():
            # compute guessed labels of unlabel samples
            outputs_u = model(inputs_u)
            outputs_u2 = model(inputs_u2)

        unlabeled_target = torch.cat([outputs_u, outputs_u2], dim=0)
        all_inputs_u = torch.cat([inputs_u, inputs_u2])
        all_outputs_u = torch.cat([outputs_u, outputs_u2], dim = 0)


        mix_input, mix_target, mix_U, mix_p = mixmatch(inputs_x, targets_x, inputs_u, inputs_u2, model)

        logits_x = model(mix_input)
        logits_u = model(mix_U)

        Lx, Lu, w = criterion(logits_x, mix_target, logits_u, mix_p)


        loss = Lx + w * Lu

        # record loss
        losses.update(loss.item(), inputs_x.size(0))
        losses_x.update(Lx.item(), inputs_x.size(0))
        losses_u.update(Lu.item(), inputs_x.size(0))
        ws.update(w, inputs_x.size(0))


        # compute gradient and do SGD step
        optimizer.zero_grad()

        loss.backward()

        optimizer.step()


        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        # plot progress

        if(batch_idx % 100 ==0):
          print('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | Loss: {loss:.4f} | Loss_x: {loss_x:.4f} | Loss_u: {loss_u:.4f}'.format(
                      batch=batch_idx + 1,
                      size=args.val_iteration,
                      data=data_time.avg,
                      bt=batch_time.avg,
                      total=bar.elapsed_td,
                      loss=losses.avg,
                      loss_x=losses_x.avg,
                      loss_u=losses_u.avg,
                      ))
          bar.next()
    bar.finish()

    return (losses.avg, losses_x.avg, losses_u.avg,)

In [0]:
# 주어진 for_student.ipynb의 validation function에 해당하는 기본 skeleton 입니다.

def validate(valloader, model, criterion, epoch, use_cuda, mode):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()
    top5 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    bar = Bar(f'{mode}', max=len(valloader))
    with torch.no_grad():
        for batch_idx, (inputs, targets) in enumerate(valloader):
            # measure data loading time
            data_time.update(time.time() - end)

            if use_cuda:
                inputs, targets = inputs.cuda(), targets.cuda(non_blocking=True)

            # compute output
            outputs = model(inputs)
            loss = criterion(outputs, targets)

            # measure accuracy and record loss
            prec1, prec5 = accuracy(outputs, targets, topk=(1, 5))
            losses.update(loss.item(), inputs.size(0))
            top1.update(prec1.item(), inputs.size(0))
            top5.update(prec5.item(), inputs.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            # plot progress
            if((batch_idx+1 ==len(valloader))):
              print('({batch}/{size}) Data: {data:.3f}s | Batch: {bt:.3f}s | Total: {total:} | Loss: {loss:.4f} | top1: {top1: .4f} | top5: {top5: .4f}'.format(
                          batch=batch_idx + 1,
                          size=len(valloader),
                          data=data_time.avg,
                          bt=batch_time.avg,
                          total=bar.elapsed_td,
                          loss=losses.avg,
                          top1=top1.avg,
                          top5=top5.avg,
                          ))
              bar.next()
        bar.finish()
    return (losses.avg, top1.avg)


In [5]:
# 주어진 for_student.ipynb의 기본 skeleton 입니다.
# Dataset을 로드해오는 기능을 수행합니다.
if not os.path.isdir(args.out):
    '''make dir if not exist'''
    try:
        os.makedirs(args.out)
    except:
        pass

# Data
print(f'==> Preparing cifar10')
transform_train = transforms.Compose([
    dataset.RandomPadandCrop(32),
    dataset.RandomFlip(),
    dataset.ToTensor(),
])

transform_val = transforms.Compose([
    dataset.ToTensor(),
])



def CustomDataLoader(labeled_count):
  train_labeled_set, train_unlabeled_set, val_set, test_set = dataset.get_cifar10('./data', labeled_count, transform_train=transform_train, transform_val=transform_val)
  return train_labeled_set, train_unlabeled_set, val_set, test_set



==> Preparing cifar10


In [0]:
# Wide_RestNet을 labeled data가 250, 1000, 4000, 10000개 있을 때의 훈련을 진행하여
# 각 case의 performance를 측정한다.

def switch_state(x):   # for 문의 index를 통해 labeled data가 몇 개가 선택되어야 하는지를 결정하는 함수
    return {
        '0': 250,
        '1': 1000,
        '2': 4000,
        '3': 10000,
        '4': 30000
    }.get(x, 50000) #default


for i in range(0, 5):
  print("{}개의 labeled data를 이용하여 training 시작\n".format(switch_state(str(i))))
  train_labeled_set, train_unlabeled_set, val_set, test_set = CustomDataLoader(int(switch_state(str(i))) )

  labeled_trainloader = data.DataLoader(train_labeled_set, 
                                        batch_size=args.batch_size, 
                                        shuffle=True, 
                                        num_workers=0, drop_last=True)

  unlabeled_trainloader = data.DataLoader(train_unlabeled_set,
                                          batch_size=args.batch_size,
                                          shuffle=True,
                                          num_workers=0, drop_last=True)

  val_loader = data.DataLoader(val_set,
                              batch_size=args.batch_size,
                              shuffle=False,
                              num_workers=0)

  test_loader = data.DataLoader(test_set, 
                                batch_size=args.batch_size,
                                shuffle=False, 
                                num_workers=0)

  ## run the model
  print("==> creating backbone network")

  model = models.WideResNet(num_classes=10)
  model = model.cuda()


  cudnn.benchmark = True
  print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

  print("==> defining loss function and optimizer")
  train_criterion = loss.SemiLoss()
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=args.lr)

  ## Training
  start_epoch = 0


  # early epoch 구현
  early_stopping_standard = 3
  early_stopping_count = 0
  temp_val_loss = 0
  best_epoch = 0
  for epoch in range(start_epoch, args.epochs):
      print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, args.epochs, args['lr']))

      train_loss, train_loss_x, train_loss_u = train(labeled_trainloader, unlabeled_trainloader, model, optimizer, train_criterion, epoch, use_cuda=True)
      _, train_acc = validate(labeled_trainloader, model, criterion, epoch, use_cuda=True, mode='Train Stats')
      val_loss, val_acc = validate(val_loader, model, criterion, epoch, use_cuda=True, mode='Valid Stats')
      test_loss, test_acc = validate(test_loader, model, criterion, epoch, use_cuda=True, mode='Test Stats ')

      if epoch == 0:    # 첫 epoch에서
        temp_val_loss = val_loss    # 임시 validation loss 변수에 이번 epoch의 validation loss를 저장한다.
      else:
        if (temp_val_loss < val_loss):   # 임시 저장해놓은 validation loss보다 이번 epoch의 validation loss가 더 클 경우
          early_stopping_count = early_stopping_count+1    # early_stopping count를 1 증가시킨다.
        else:
          best_epoch = epoch+1                      # 만일 validation loss가 작아졌다면, best_epoch 기록을 남기고
          temp_val_loss = val_loss                  # 임시 validation loss를 초기화하고
          early_stopping_count = 0                  # early_stopping count를 초기화한다.

      if(early_stopping_count == early_stopping_standard):           # early_stopping count가 제한 횟수와 같아질 경우
        print("최적 epoch : {}\n".format(best_epoch))               # print하고 학습을 종료한다.
        break;


In [0]:
# U_net model 정의
# https://github.com/usuyama/pytorch-unet/blob/master/pytorch_unet.py 에 기재된 U_net code를 일부
# 수정하여 사용하였습니다.
# CIFAR10 훈련에 사용하기 위해 output layer를 변경하였습니다.

def double_conv(in_channels, out_channels):
    return nn.Sequential(
        nn.Conv2d(in_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(out_channels, out_channels, 3, padding=1),
        nn.ReLU(inplace=True)
    )   


class UNet(nn.Module):

    def __init__(self):
        super().__init__()
                
        self.dconv_down1 = double_conv(3, 64)
        self.dconv_down2 = double_conv(64, 128)
        self.dconv_down3 = double_conv(128, 256)
        self.dconv_down4 = double_conv(256, 512)        

        self.maxpool = nn.MaxPool2d(2)
        self.upsample = nn.Upsample(scale_factor=2, mode='bilinear', align_corners=True)        
        
        self.dconv_up3 = double_conv(256 + 512, 256)
        self.dconv_up2 = double_conv(128 + 256, 128)
        self.dconv_up1 = double_conv(128 + 64, 64)
        
        self.conv_last = nn.Conv2d(64, 1, 1)
        self.fc = nn.Linear(32*32, 100)
        self.fc2 = nn.Linear (100, 10)
        
        
    def forward(self, x):
        conv1 = self.dconv_down1(x)
        x = self.maxpool(conv1)

        conv2 = self.dconv_down2(x)
        x = self.maxpool(conv2)
        
        conv3 = self.dconv_down3(x)
        x = self.maxpool(conv3)   
        
        x = self.dconv_down4(x)
        
        x = self.upsample(x)        
        x = torch.cat([x, conv3], dim=1)
        
        x = self.dconv_up3(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv2], dim=1)       

        x = self.dconv_up2(x)
        x = self.upsample(x)        
        x = torch.cat([x, conv1], dim=1)   
        
        x = self.dconv_up1(x)
        
        x = self.conv_last(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        out = self.fc2(x)
        
        return out

In [0]:
"""
위와 동일하게 학습 알고리즘을 가지나,
model만 Unet으로 교체하여 진행하였다.
"""
torch.cuda.empty_cache()

def switch_state(x):
    return {
        '0': 250,
        '1': 1000,
        '2': 4000,
        '3': 10000,
        '4': 30000
    }.get(x, 50000) #default

for i in range(0, 5):
  print("{}개의 labeled data를 이용하여 training 시작\n".format(switch_state(str(i))))
  train_labeled_set, train_unlabeled_set, val_set, test_set = CustomDataLoader(int(switch_state(str(i))) )

  labeled_trainloader = data.DataLoader(train_labeled_set, 
                                        batch_size=args.batch_size, 
                                        shuffle=True, 
                                        num_workers=0, drop_last=True)

  unlabeled_trainloader = data.DataLoader(train_unlabeled_set,
                                          batch_size=args.batch_size,
                                          shuffle=True,
                                          num_workers=0, drop_last=True)

  val_loader = data.DataLoader(val_set,
                              batch_size=args.batch_size,
                              shuffle=False,
                              num_workers=0)

  test_loader = data.DataLoader(test_set, 
                                batch_size=args.batch_size,
                                shuffle=False, 
                                num_workers=0)

  ## run the model
  print("==> creating backbone network")

  model = UNet()
  model = model.cuda()


  cudnn.benchmark = True
  print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

  print("==> defining loss function and optimizer")
  train_criterion = loss.SemiLoss()
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)

  ## Training
  start_epoch = 0


  # early epoch 구현
  early_stopping_standard = 3
  early_stopping_count = 0
  temp_val_loss = 0
  best_epoch = 0
  for epoch in range(start_epoch, args.epochs):
      print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, args.epochs, args['lr']))

      train_loss, train_loss_x, train_loss_u = train(labeled_trainloader, unlabeled_trainloader, model, optimizer, train_criterion, epoch, use_cuda=True)
      _, train_acc = validate(labeled_trainloader, model, criterion, epoch, use_cuda=True, mode='Train Stats')
      val_loss, val_acc = validate(val_loader, model, criterion, epoch, use_cuda=True, mode='Valid Stats')
      test_loss, test_acc = validate(test_loader, model, criterion, epoch, use_cuda=True, mode='Test Stats ')

      if epoch == 0:
        temp_val_loss = val_loss
      else:
        if (temp_val_loss < val_loss):
          early_stopping_count = early_stopping_count+1
        else:
          best_epoch = epoch+1
          temp_val_loss = val_loss
          early_stopping_count = 0

      if(early_stopping_count == early_stopping_standard):
        print("최적 epoch : {}\n".format(best_epoch))
        break;

In [0]:
"""
위와 동일하게 학습 알고리즘을 가지나,
model만 resnet18로 변경하여 학습하였다.
"""
import torchvision
torch.cuda.empty_cache()

def switch_state(x):
    return {
        '0': 250,
        '1': 1000,
        '2': 4000,
        '3': 10000,
        '4': 30000
    }.get(x, 50000) #default

for i in range(0, 5):
  print("{}개의 labeled data를 이용하여 training 시작\n".format(switch_state(str(i))))
  train_labeled_set, train_unlabeled_set, val_set, test_set = CustomDataLoader(int(switch_state(str(i))) )

  labeled_trainloader = data.DataLoader(train_labeled_set, 
                                        batch_size=args.batch_size, 
                                        shuffle=True, 
                                        num_workers=0, drop_last=True)

  unlabeled_trainloader = data.DataLoader(train_unlabeled_set,
                                          batch_size=args.batch_size,
                                          shuffle=True,
                                          num_workers=0, drop_last=True)

  val_loader = data.DataLoader(val_set,
                              batch_size=args.batch_size,
                              shuffle=False,
                              num_workers=0)

  test_loader = data.DataLoader(test_set, 
                                batch_size=args.batch_size,
                                shuffle=False, 
                                num_workers=0)

  ## run the model
  print("==> creating backbone network")

  model = torchvision.models.resnet18(pretrained=False)
  model.fc = nn.Linear(512,10)
  model = model.cuda()


  cudnn.benchmark = True
  print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

  print("==> defining loss function and optimizer")
  train_criterion = loss.SemiLoss()
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.01)

  ## Training
  start_epoch = 0


  # early epoch 구현
  early_stopping_standard = 3
  early_stopping_count = 0
  temp_val_loss = 0
  best_epoch = 0
  for epoch in range(start_epoch, args.epochs):
      print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, args.epochs, args['lr']))

      train_loss, train_loss_x, train_loss_u = train(labeled_trainloader, unlabeled_trainloader, model, optimizer, train_criterion, epoch, use_cuda=True)
      _, train_acc = validate(labeled_trainloader, model, criterion, epoch, use_cuda=True, mode='Train Stats')
      val_loss, val_acc = validate(val_loader, model, criterion, epoch, use_cuda=True, mode='Valid Stats')
      test_loss, test_acc = validate(test_loader, model, criterion, epoch, use_cuda=True, mode='Test Stats ')

      if epoch == 0:
        temp_val_loss = val_loss
      else:
        if (temp_val_loss < val_loss):
          early_stopping_count = early_stopping_count+1
        else:
          best_epoch = epoch+1
          temp_val_loss = val_loss
          early_stopping_count = 0

      if(early_stopping_count == early_stopping_standard):
        print("최적 epoch : {}\n".format(best_epoch))
        break;

In [0]:
# 위의 Wide-ResNet training과 동일하나, Epoch와 Early stopping 기준 count 를 증가시키고 performance를 체크하였다.
# 또한 learning rate를 감소시켰다.
torch.cuda.empty_cache()
def switch_state(x):
    return {
        '0': 250,
        '1': 1000,
        '2': 4000,
        '3': 10000,
        '4': 30000
    }.get(x, 50000) #default


for i in range(0, 5):
  print("{}개의 labeled data를 이용하여 training 시작\n".format(switch_state(str(i))))
  train_labeled_set, train_unlabeled_set, val_set, test_set = CustomDataLoader(int(switch_state(str(i))) )

  labeled_trainloader = data.DataLoader(train_labeled_set, 
                                        batch_size=args.batch_size, 
                                        shuffle=True, 
                                        num_workers=0, drop_last=True)

  unlabeled_trainloader = data.DataLoader(train_unlabeled_set,
                                          batch_size=args.batch_size,
                                          shuffle=True,
                                          num_workers=0, drop_last=True)

  val_loader = data.DataLoader(val_set,
                              batch_size=args.batch_size,
                              shuffle=False,
                              num_workers=0)

  test_loader = data.DataLoader(test_set, 
                                batch_size=args.batch_size,
                                shuffle=False, 
                                num_workers=0)

  ## run the model
  print("==> creating backbone network")

  model = models.WideResNet(num_classes=10)
  model = model.cuda()


  cudnn.benchmark = True
  print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

  print("==> defining loss function and optimizer")
  train_criterion = loss.SemiLoss()
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001)

  ## Training
  start_epoch = 0


  # early epoch 구현
  early_stopping_standard = 5
  early_stopping_count = 0
  temp_val_loss = 0
  best_epoch = 0
  for epoch in range(start_epoch, 50):
      print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, 50, 0.001))

      train_loss, train_loss_x, train_loss_u = train(labeled_trainloader, unlabeled_trainloader, model, optimizer, train_criterion, epoch, use_cuda=True)
      _, train_acc = validate(labeled_trainloader, model, criterion, epoch, use_cuda=True, mode='Train Stats')
      val_loss, val_acc = validate(val_loader, model, criterion, epoch, use_cuda=True, mode='Valid Stats')
      test_loss, test_acc = validate(test_loader, model, criterion, epoch, use_cuda=True, mode='Test Stats ')

      if epoch == 0:
        temp_val_loss = val_loss
      else:
        if (temp_val_loss < val_loss):
          early_stopping_count = early_stopping_count+1
        else:
          best_epoch = epoch+1
          temp_val_loss = val_loss
          early_stopping_count = 0

      if(early_stopping_count == early_stopping_standard):
        print("최적 epoch : {}\n".format(best_epoch))
        break;

In [0]:
# 위 Wide-ResNet에 3가지 변화를 준 모델에 L2 Legularization을 위해 weight decay를 추가하였다
# decay 상수값은 0.001로 설정하였다. 
torch.cuda.empty_cache()
def switch_state(x):
    return {
        '0': 250,
        '1': 1000,
        '2': 4000,
        '3': 10000,
        '4': 30000
    }.get(x, 50000) #default


for i in range(0, 5):
  print("{}개의 labeled data를 이용하여 training 시작\n".format(switch_state(str(i))))
  train_labeled_set, train_unlabeled_set, val_set, test_set = CustomDataLoader(int(switch_state(str(i))) )

  labeled_trainloader = data.DataLoader(train_labeled_set, 
                                        batch_size=args.batch_size, 
                                        shuffle=True, 
                                        num_workers=0, drop_last=True)

  unlabeled_trainloader = data.DataLoader(train_unlabeled_set,
                                          batch_size=args.batch_size,
                                          shuffle=True,
                                          num_workers=0, drop_last=True)

  val_loader = data.DataLoader(val_set,
                              batch_size=args.batch_size,
                              shuffle=False,
                              num_workers=0)

  test_loader = data.DataLoader(test_set, 
                                batch_size=args.batch_size,
                                shuffle=False, 
                                num_workers=0)

  ## run the model
  print("==> creating backbone network")

  model = models.WideResNet(num_classes=10)
  model = model.cuda()


  cudnn.benchmark = True
  print('    Total params: %.2fM' % (sum(p.numel() for p in model.parameters())/1000000.0))

  print("==> defining loss function and optimizer")
  train_criterion = loss.SemiLoss()
  criterion = nn.CrossEntropyLoss()
  optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay = 0.001)  # weight decay 적용

  ## Training
  start_epoch = 0


  # early epoch 구현
  early_stopping_standard = 5
  early_stopping_count = 0
  temp_val_loss = 0
  best_epoch = 0
  for epoch in range(start_epoch, 50):
      print('\nEpoch: [%d | %d] LR: %f' % (epoch + 1, 50, 0.001))

      train_loss, train_loss_x, train_loss_u = train(labeled_trainloader, unlabeled_trainloader, model, optimizer, train_criterion, epoch, use_cuda=True)
      _, train_acc = validate(labeled_trainloader, model, criterion, epoch, use_cuda=True, mode='Train Stats')
      val_loss, val_acc = validate(val_loader, model, criterion, epoch, use_cuda=True, mode='Valid Stats')
      test_loss, test_acc = validate(test_loader, model, criterion, epoch, use_cuda=True, mode='Test Stats ')

      if epoch == 0:
        temp_val_loss = val_loss
      else:
        if (temp_val_loss < val_loss):
          early_stopping_count = early_stopping_count+1
        else:
          best_epoch = epoch+1
          temp_val_loss = val_loss
          early_stopping_count = 0

      if(early_stopping_count == early_stopping_standard):
        print("최적 epoch : {}\n".format(best_epoch))
        break;